# QLD  Sensitive Species List

This notebook downloads the Qld lists from the [Qld Government Open Data Portal](https://data.qld.gov.au) and formats them in Darwin Core for ingestion into the ALA Lists tool.
It will save original lists to the `source-data/QLD` directory, process the lists and save them to `current-lists`.


## Lists in the ALA Species List tool
* Sensitive list: __[dr493](https://lists.ala.org.au/speciesListItem/list/dr493)__ ([dr18404 in test](https://lists-test.ala.org.au/speciesListItem/list/dr18404))
* Collection: __[dr493 in test](https://collections.ala.org.au/public/show/dr493)__

## Source Data
Queensland Nature Conservation Act 1992

* __[Sensitive](https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/qld-confidential-species.csv)__
* __[Species Codes](https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species-status-codes.csv)__

**Metadata Description**

**Sensitive:** A list of confidential species for Queensland or "sensitive species" supplied directly by the Queensland Department of Environment and Science. It is published weekly as the Queensland Confidential Species dataset in the Queensland Government Data portal. The Atlas of Living Australia generalises all latitude and longitude values in occurrence records of sensitive species in Queensland to one decimal place. This rule is applied to all Queensland occurrence records regardless of source.

**Metadata URL**
* Qld Species (Open Data Portal) https://www.data.qld.gov.au/dataset/conservation-status-of-queensland-wildlife
* Queensland Confidential Species (Open Data Portal) https://www.data.qld.gov.au/dataset/queensland-confidential-species
* Qld Species codes https://www.data.qld.gov.au/dataset/conservation-status-of-queensland-wildlife/resource/6344ea93-cadf-4e0c-9ff4-12dfb18d5f14


# Setup
* Import libraries
* Set Project directory
* Set URLs

In [8]:
import pandas as pd
import requests
import io
from ftfy import fix_encoding
import os
import sys
import datetime

projectDir = "/Users/oco115/PycharmProjects/authoritative-lists/"
# projectDir = "/Users/new330/IdeaProjects/authoritative-lists/"
sys.path.append(os.path.abspath(projectDir + "source-code/includes"))

sourceDataDir = projectDir + "source-data/QLD/"
statusDir = projectDir + "source-data/status-codes/"
processedDataDir = projectDir + "current-lists/"
state = 'QLD'
monthStr = datetime.datetime.now().strftime('%Y%m')
codesfile = statusDir + state + "-codes.csv"
codesurl =  "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species-status-codes.csv"
sensitivelisturl = "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/qld-confidential-species.csv"

## Download the raw files from data.qld.gov.au
... save locally

In [ ]:
# Status codes
response = requests.get(codesurl)
rtext = fix_encoding(response.text)
speciescodes = pd.read_csv(io.StringIO(rtext))
speciescodes.to_csv(sourceDataDir + "species-status-codes.csv", index=False)

# Confidential/Sensitive List
response = requests.get(sensitivelisturl)
rtext = fix_encoding(response.text)
sensitivelist = pd.read_csv(io.StringIO(rtext))
sensitivelist.to_csv(sourceDataDir + "qld-confidential-species.csv", index=False)

## Standardise Status Codes
Some minimal changes to some Qld Nature Conservation Act codes so that they are consistent with other states

In [69]:
speciescodes = pd.read_csv(sourceDataDir + "species-status-codes.csv")
ncastatuscodes = speciescodes[speciescodes['Field'] == "NCA_status"][['Code', 'Code_description']]
ncastatuscodes['Code_description'] = ncastatuscodes['Code_description'].str.replace(" wildlife", "")
ncastatuscodes.loc[
    ncastatuscodes['Code_description'] == "Critically endangered", 'Code_description'] = "Critically Endangered"
ncastatuscodes.loc[ncastatuscodes['Code_description'] == "Near threatened", 'Code_description'] = "Near Threatened"
ncastatuscodes

,Code,Code_description
20,C,Least concern
21,CR,Critically Endangered
22,E,Endangered
23,EX,Extinct
24,I,International
25,NT,Near Threatened
26,P,Prohibited
27,PE,Extinct in the wild
28,SL,Special least concern
29,V,Vulnerable


## Option to read from local files for subsequent runs

In [78]:
sensitivelist = pd.read_csv(sourceDataDir + "qld-confidential-species.csv")

## Sensitive - Qld Confidential list
* Read in the Confidential/Sensitive list
* Expand the nca status, endemicity and epbc status codes
* Rename fields to DwC terms
* Replace kingdom and class values with scientific terms


In [79]:
# nca status
sensitivelist = pd.merge(sensitivelist,ncastatuscodes,left_on=['NCA status'],right_on=['Code'],how="left")
sensitivelist = sensitivelist.rename(columns={'NCA status':'sourceStatus'})
sensitivelist = sensitivelist[['Scientific name', 'Common name', 'sourceStatus']]
sensitivelist = sensitivelist.rename(columns= {'Scientific name':'scientificName',
                                               'Common name': 'vernacularName',
                                                'sourceStatus': 'category' })
sensitivelist['generalisation'] = "2 km"

In [80]:
# map sourceStatus to category
codeMap = {'C': 'LC', 'CR': 'CR', 'E': 'EN',
           'NT': 'NT','PE': 'EW', 'SL': 'SL',
           'V': 'VU'}
sensitivelist['category'] = sensitivelist['category'].replace(codeMap)
sensitivelist['category'] = sensitivelist['category'].fillna('UK')
sensitivelist

,scientificName,vernacularName,category,generalisation
0,Rhinolophus philippinensis,greater large-eared horseshoe bat,EN,2 km
1,Chloebia gouldiae,Gouldian finch,EN,2 km
2,Erythrura trichroa,blue-faced parrot-finch,NT,2 km
3,Neochmia phaeton evangelinae,crimson finch (white-bellied subspecies),EN,2 km
4,Poephila cincta cincta,black-throated finch (white-rumped subspecies),EN,2 km
...,...,...,...,...
954,Pneumatopteris pennigera,lime fern,EN,2 km
955,Reholttumia costata,NaN,NT,2 km
956,Thelypteris confluens,NaN,VU,2 km
957,Macadamia jansenii,NaN,CR,2 km


## Write to CSV

In [ ]:
sensitivelist.to_csv(processedDataDir + "sensitive-lists/QLD-sensitive-20230614.csv",encoding="UTF-8",index=False)
print("Finished writing QLD Sensitive list")

In [ ]:
### Change logs

In [ ]:
ltype = "S"
changeDir = "Monitoring/Change-logs/"
# Qld Sensitive
print('QLD sensitive')
filename = "QLD-sensitive.csv"
testdr = "dr18404"
proddr = "dr493"
changelist = lf.get_changelist(testdr, proddr, ltype)
changelist.to_csv(projectDir + changeDir + monthStr + "-" + filename, encoding="UTF-8", index=False)
print('Finished QLD sensitive')

In [ ]:
# Sensitive Lists

filename = state + "-" + proddr + "-sensitive.csv"
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + proddr + "?max=10000&includeKVP=true"
prodList = lf.download_ala_list(prodListUrl)  # save the prod list to the historical lists directory
prodList = lf.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/sensitive/" + filename, encoding="UTF-8", index=False)
print('Finished downloading sensitive historical list')